In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()


[<Experiment: artifact_location='/workspaces/MLOps_project/02-experiment_tracking/mlruns/1', creation_time=1719822930781, experiment_id='1', last_update_time=1719822930781, lifecycle_stage='active', name='house-pricing-experiment', tags={}>]

In [2]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse >1",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)


In [3]:

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: ebec85ece752449596f1743f425ebf99, rmse: 1335981.0995
run id: 83a6cba7c4f34d7f967f68828a220c9b, rmse: 1345497.3778
run id: da4bbea9fa9246a2a7c67cda05ea8b86, rmse: 1349097.9984
run id: ba1c744299fa4497835a99c35f79d291, rmse: 1376843.6824
run id: 36f3cdaa79b54e22a37763c876fc82f5, rmse: 1376843.6824


In [4]:
# Interacting with the Model Registry

import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "4f4244ef75764938ad5bd8d08f8642ab"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="House-pricing-model")

Registered model 'House-pricing-model' already exists. Creating a new version of this model...
Created version '3' of model 'House-pricing-model'.


<ModelVersion: aliases=[], creation_timestamp=1719933601243, current_stage='None', description=None, last_updated_timestamp=1719933601243, name='House-pricing-model', run_id='4f4244ef75764938ad5bd8d08f8642ab', run_link=None, source='/workspaces/MLOps_project/02-experiment_tracking/mlruns/1/4f4244ef75764938ad5bd8d08f8642ab/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [5]:

model_name = "House-pricing-model"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None
version: 2, stage: Production


/tmp/ipykernel_38554/2232596705.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [22]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_4474/4090915774.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1719860009520, current_stage='Staging', description=None, last_updated_timestamp=1719860072734, name='House-pricing-model', run_id='ebec85ece752449596f1743f425ebf99', run_link=None, source='/workspaces/MLOps_project/02-experiment_tracking/mlruns/1/ebec85ece752449596f1743f425ebf99/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [23]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1719860009520, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-07-01', last_updated_timestamp=1719860417676, name='House-pricing-model', run_id='ebec85ece752449596f1743f425ebf99', run_link=None, source='/workspaces/MLOps_project/02-experiment_tracking/mlruns/1/ebec85ece752449596f1743f425ebf99/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [24]:
model_version = 2
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_4474/3059153048.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1719860009520, current_stage='Production', description='The model version 2 was transitioned to Staging on 2024-07-01', last_updated_timestamp=1719861299532, name='House-pricing-model', run_id='ebec85ece752449596f1743f425ebf99', run_link=None, source='/workspaces/MLOps_project/02-experiment_tracking/mlruns/1/ebec85ece752449596f1743f425ebf99/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>